In [10]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

print(f"OpenAI API Key loaded: {api_key is not None}")

OpenAI API Key loaded: True


In [19]:
from openai import OpenAI

client = OpenAI(api_key = api_key)

def ask_llm(question: str) ->str:
    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = [
            {"role": "user", "content": question},
            {"role": "system", "content": "You are a helpful assistant."}
        ],
        temperature = 0.7,
        max_tokens = 150
    )
    return response.choices[0].message.content
print("LLM is ready to answer your questions!")

LLM is ready to answer your questions!


In [20]:
question = "What is FastAPI?"
answer = ask_llm(question)
print(f"Question: {question}\n")
print(f"Answer: {answer}")

Question: What is FastAPI?

Answer: FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints. It is designed to be simple, intuitive, and easy to use, while also being highly efficient and scalable. FastAPI is built on top of Starlette for the web parts and Pydantic for the data parts, making it a powerful and feature-rich framework for building web applications and APIs.


In [22]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(f"OpenAI API KEy loaded: {api_key is not None}")

OpenAI API KEy loaded: True


In [27]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

def ask_llm(question:str) ->str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = [
            {"role": "user", "content": question},
            {"role": "system", "content": "You are a helpful assistant."}
        ],
        temperature = 0.7,
        max_tokens=100
    )
    return response.choices[0].message.content
print("LLM is ready to answer your questions!")

LLM is ready to answer your questions!


In [29]:
question = "What is the capital of France?"
answer = ask_llm(question)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

Question: What is the capital of France?

Answer: The capital of France is Paris.


# Using Langchain

In [36]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings

from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI
from langchain_classic.chains.retrieval_qa.base import RetrievalQA

from typing import List
import glob

In [38]:
loader = PyPDFLoader("Cover_Letter.pdf")
documents = loader.load()

print(f"Length of documents: {len(documents)} docs")
print(f"Contents of first doc: {documents[0].page_content[:40]}..")
print(f"metadata of first doc: {documents[0].metadata}")

Length of documents: 1 docs
Contents of first doc: Prakyath Chandran 
Bangalore, India 
cha..
metadata of first doc: {'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2026-01-06T09:15:40+05:30', 'author': 'Un-named', 'moddate': '2026-01-06T09:15:40+05:30', 'source': 'Cover_Letter.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}


In [39]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function=len,
    separators = ["\n\n", "\n", " ", ""]
)

chunks = splitter.split_documents(documents)

print(f"Number of chunks created: {len(chunks)}")

Number of chunks created: 2


In [41]:
embeddings = HuggingFaceEmbeddings(
    model_name = "all-MiniLM-L6-v2",
)

test_text = "What is the name of the applicant?"
test_embeding = embeddings.embed_query(test_text)

print(f"Embedding vector length/Dimension: {len(test_embeding)}")

Embedding vector length/Dimension: 384


In [43]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="cover_letter_docs"
)

print("Vector store created successfully!")
print(f"Number of documents in vector store: {vectorstore._collection.count()}")

Vector store created successfully!
Number of documents in vector store: 2


In [46]:
query = "What role is applicant applying for??"
results = vectorstore.similarity_search(query, k=1)

print(f"Query: {query}\n")
for i, doc in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"{doc.page_content[:150]}...")
    print(f"Source: Page {doc.metadata['page']}\n")

Query: What role is applicant applying for??

Result 1:
At HaiX AI, I benchmarked 5+ LLMs systematically, testing 100+ prompt variations and 
identifying cost-efficiency opportunities. I built FastAPI endpo...
Source: Page 0



In [47]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=150,
    api_key= os.environ["OPENAI_API_KEY"]
)

response = llm.invoke("What is applicant applying for in the cover letter?")
print(f"Response from LLM: {response.content}")

Response from LLM: The applicant is applying for a specific job or position at a company or organization. They may mention the job title or department they are interested in, and highlight their qualifications and experience that make them a strong candidate for the role.


In [48]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(search_kwargs={"k":1}),
    return_source_documents=True
)

print("RAG chain is set up and ready to use!")

RAG chain is set up and ready to use!


In [52]:
question = "What is the tone of the cover letter?"
result = rag_chain.invoke(question)

print(f"Question: {question}\n")
print(f"Answer:\n{result['result']}\n")
print("="*80)
print(f"Sources ({len(result['source_documents'])} chunks):")
for i, doc in enumerate(result['source_documents'], 1):
    print(f"\n{i}. Page {doc.metadata['page']}:")
    print(f"   {doc.page_content[:50]}...")

Question: What is the tone of the cover letter?

Answer:
The tone of the cover letter is professional and enthusiastic.

Sources (1 chunks):

1. Page 0:
   Prakyath Chandran 
Bangalore, India 
chandranpraky...
